# Load Libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Data science
import math
import scipy.stats as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.stats.multitest import multipletests as mt

# Plots
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import to_hex

# Working with dates
from datetime import datetime
from datetime import date
import dateutil

# Looping  progress
from tqdm.notebook import tqdm

# Reg expressions
import re

# Pretty table printing
import tabulate

# ***REMOVED*** Snippets Require these
import os
import subprocess

# Misc libraries
from IPython.display import display, HTML


# Set seaborn figure size, font size, and style
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set(font_scale=1.5)
sns.set_style("white")

# Set Pandas options so we can see our entire dataframe
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
pd.options.display.max_colwidth = None

# Print our versions of this packages, this allows us to make sure
# we have the working versions we need. 
print(f"Pandas version: {pd.__version__}")

import dask.dataframe as dd

In [ ]:
BASE_DIR = "***REMOVED***/other/trinetx/new_dataset"
HOME = "***REMOVED***"

# General data manipulation

Because a lot of these files were large, a lot of the initial handling happened using BASH and different CLI tools, especially GNU Parallel. I'll try to document everything that was done here.

## Decrypting and Decompressing


```bash
# Move to our main directory
cd $BASE_DIR

# Assume zip file from TNX is called dataset_abc.zip
unzip dataset_abc.zip

mv dataset_abc new_dataset
```

# Diagnoses

## Exploring Diagnosis data

* The first step is to break the ICD codes up into 3-character and post 'decimal' bit. --pipepart will break the file input to GNU parallel (-a flag) into equal chunks based on the block size.

* The AWK command needs to survive GNU parallel's argument processing and still be surrounded by single quotes, so an extra layer of quotes (and some escaping) is required.
 
* The AWK program itself is splitting the 4th column, the ICD code on the decimal and then reprinting the original line plus a new column for the 3-char ICD code and a new column for the post-decimal section.

```bash
# Move into our directory
BASE_DIR="${BASE_DIR}/new_dataset"
cd "${BASE_DIR}"

# Call the parallel program
parallel --jobs 16 --block 1G --pipepart -a diagnosis.csv  awk -F, -vOFS=, "'{ split(\$4, a, \".\"); print \$0, a[1], a[2]}'" > diagnosis_3_char.csv
```
  
  
After splitting out the 3-character ICD code we want to sort by the patient ID (column 1), the 3-character ICD code (column 11), and the date (column 8), so that all diagnoses for each patient are sorted within patient and within ICD code by date, with the oldest diagnosis instance at the top of the file.

```bash
# Parsort is written by Ole Tange and attached GNU parallel and sort
parsort --parallel=16 -t ','  -k1,1 -k11,11 -k8,8 diagnosis_3_char.csv > diagnosis_3_char_sorted.csv
```

Now to do some exploration, I wanted to look at the distribution of a few values
* First just looking at the source ID 

```bash
awk -F, '{print $10}' diagnosis_3_char_sorted.csv | sort | uniq -c > src_count

head src_count | column -t
1861535170  "EHR"
300089434   "NLP"
1           "source_id"
253158      "TriNetX"
```

So it looks like most of the diagnoses are straight from the EHR, but 300M are from NLP so we need some kind of rules here. For now we will only use EHR data.
* EHR_ONLY: Only include EHR diagnoses
* ALL_SRC: Include diagnoses from all sources
* EHR_DATE_FIRST: Include diagnoses from all sources but use the EHR date of diagnosis for each disease, unless there is not an EHR diagnosis for that code in which case, use the earliest date, either from NLP or TriNetX.

```bash
# Only print out columns where the 10th column (the source column) is EHR.
awk -F, '$10 == "\"EHR\""' diagnosis_3_char_sorted.csv > diagnosis_3_char_sorted_ehr_only.csv
```

I also noted there are more than just ICD10 codes, so I wanted to investigate what we have there. It looks like just under 68% of all diagnoses are in ICD10, which is good because that's all we have codes for in UKB without doing some sort of crazy translation, which isn't super straightforward unfortunately. So for now we will be ignoring the ICD9 codes.

```bash
# Get the counts for each type of ICD coding scheme

parallel --jobs 16 --block 1G --pipepart -a diagnosis_3_char_sorted_ehr_only.csv \
         awk -F, "'{print \$3}'" > icd_code.list  &&
  parsort --parallel=16 -t, icd_code.list > icd_code_sorted.list &&
  uniq -c icd_code_sorted.list > icd_code_counts &&
  sort -k1 -nr icd_code_counts | sponge icd_code_counts

# 1262909432 "ICD-10-CM"
# 598622778 "ICD-9-CM"
#   2960 ""
```
  

## Splitting diagnoses into 3-char

The simplest way I could think to further process the diagnosis data was to split it into much more manageable chunks that we could read and process with Python. To do this, I figured it would be easiest to just split each of the ICD10 codes we are interested in from UKB into a separate file. To do this I'll be using a [submitArrayJobs](https://github.com/ernstki/submitArrayJobs) call.

<hr>
Create the break_diags.sh script to filter for each ICD10 code listed in the array file, as we will be using break_diags.sh as our command file.

```bash
cat break_diags.sh

#!/bin/bash

OUT_DIR="${BASE_DIR}/icd_data"

# ICD code from array file inserted by submitArrayJobs prior to be submitted to HPC.
curr_icd=VAR1
curr_fn="${OUT_DIR}/${curr_icd}_only.csv"


# Need to export so GNU Parallel can see it
export curr_icd


parallel --jobs 16 --block 1G --pipepart -a diagnosis_3_char_sorted.csv  awk -v icd=\"\$curr_icd\" -F\",\" \'\$11 \~ icd \{print \$0\;\}\' > "${curr_fn}"
```

<hr>

The array file is simply a list of the ICD codes we are interested in:
```bash
head af
A60     A60
B02     B02
B19     B19
B24     B24
...
```

<hr>

Then we can just submit these jobs using submitArrayJobs

```bash
submitArrayJobs -C break_diags.sh -A af -W 24:00 -n 16 -M 200000

```

# Lab data

Here I'll show how we processed all the lab data, both categorical and continuous, but it's important to note that we decided to drop the continuous tests and only use categorical. We will again use submitArrayJobs (sAJ) for submitting a separate job to look through the huge (~700 GB) file of lab tests for each LOINC code of interest.


```bash
# break_labs_new.sh
#!/bin/bash

OUT_DIR="${BASE_DIR}/lab_data"

# sAJ will insert the proper LOINC code in curr_loinc
curr_loinc=VAR1
curr_fn="${OUT_DIR}/${curr_loinc}_only_single_thread.csv"

# Need to export so GNU Parallel can see it
export curr_loinc

awk -v loinc="$curr_loinc" -F"," '$4 ~ loinc {print $0;}' lab_result.csv   > "${curr_fn}"

```

<hr>


```bash
head labs_af
loinc_63475-8   63475-8
loinc_26645-2   26645-2
loinc_62466-8   62466-8
loinc_33978-8   33978-8
loinc_90924-2   90924-2
...
```

<hr>

```bash
submitArrayJobs -C break_labs_new.sh -A labs_af -W 24:00 -n 16 -M 200000
````

## Generate file that summarizes the results we have for all of the labs

In [ ]:
import glob
import csv

meta_dir = BASE_DIR
lab_counts = pd.read_csv(f"{meta_dir}/clean_loinc_counts.tsv", sep = '\t')

lab_dir = f"{BASE_DIR}/lab_data"
lab_files = glob.glob(f"{lab_dir}/*.csv")

REQ_LAB_NUM = 20

res_ls = []

for curr_lab_ind in tqdm(range(0, len(lab_files), 1)):
    curr_lab_fn = lab_files[curr_lab_ind]
    fn_loinc = os.path.basename(curr_lab_fn).split('_')[0]

    curr_dat = pd.read_csv(curr_lab_fn, sep = ',', 
                           names = ['pat_id', 'enc_id', 'LOINC', 'code', 'date', 
                                     'num_value', 'text_value', 'units', 'Tri_deriv',
                                      'src_id'], 
                           index_col=False, quoting=csv.QUOTE_NONE)
    curr_dat = curr_dat.applymap(lambda x: str(x).lstrip('"').rstrip('"'))

    
    curr_dat = curr_dat.loc[curr_dat['code'] == fn_loinc, :]
    
    if len(curr_dat) < REQ_LAB_NUM:
        print(f"{fn_loinc}: only {len(curr_dat)} people, skipping analysis!")
        continue

    curr_loinc = curr_dat['code'].unique()[0]

    if fn_loinc != curr_loinc:
        print(f"{fn_loinc}: LOINC mismatch, fn: {fn_loinc}, data: {curr_loinc}, skipping analysis!")


    test_info = lab_counts.loc[lab_counts['loinc'] == curr_loinc, :]

    n_uniq_pats = test_info['count'].tolist()[0]

    if n_uniq_pats < REQ_LAB_NUM:
        print(f"{curr_loinc}: only {n_uniq_pats} people, skipping analysis!")
        continue


    test_scale = test_info['SCALE_TYP'].tolist()[0]

    units = test_info['unit'].tolist()[0]

    if (test_scale == 'Ord') and (units != 'boolean'):
        print(f"{curr_loinc}: 'Ord' test with non-boolean units!")

    nrow = len(curr_dat)
    n_pats = len(curr_dat.loc[:, 'pat_id'].unique().tolist())

    units_dict = curr_dat['units'].value_counts().to_dict()

    non_null = curr_dat.loc[((curr_dat['num_value'].notnull()) &
                          (curr_dat['num_value'] != '') &
                          (curr_dat['num_value'] != 'nan')), :].copy(deep = True)


    n_null = len(curr_dat.loc[((curr_dat['num_value'].isnull()) |
                             (curr_dat['num_value'] == '') |
                              (curr_dat['num_value'] == 'nan')), :])
    
    non_null.loc[:, 'num_value'] = non_null.loc[:, 'num_value'].astype('float')
    

    n_vals_0 = sum(non_null['num_value'] == 0)
    #print(f"{curr_loinc}: {n_vals_0}")
    n_vals_non_0 = sum(non_null['num_value'] != 0)




    n_unique = len(non_null['num_value'].unique())

    #curr_val = non_null['num_value'].value_counts(dropna = False).to_dict()


    num_lt_0 = len(non_null.loc[non_null['num_value'] < 0, :])
    num_mean = non_null['num_value'].mean()
    num_sd = non_null['num_value'].std()
    num_med = non_null['num_value'].median()
    num_min = non_null['num_value'].min()
    num_max = non_null['num_value'].max()

    non_null_no_zero = non_null.loc[non_null['num_value'] > 0, :]
    if len(non_null_no_zero) > 0:
        bins = [0, 1, 10, 100, 1000, 10000, 1e5, 1e7, 1e8]

        tens = pd.DataFrame(non_null_no_zero['num_value'].value_counts(bins = bins, sort = False)).reset_index()
        tens_str = tens.to_string(header = False, index = False)



        num_vals = pd.DataFrame(non_null_no_zero['num_value'].value_counts(bins = 20, sort = False)).reset_index()
        num_vals_str = num_vals.to_string(header = False, index = False)
    else:
        tens_str = ''
        num_vals_str = ''


    n_cat_with_value = len(curr_dat.loc[((curr_dat['text_value'].notnull()) & 
                            (curr_dat['text_value'] != '')), 'pat_id'].unique().tolist())

    non_null = curr_dat.loc[((curr_dat['text_value'].notnull()) & 
                            (curr_dat['text_value'] != '')), :]

    curr_val_con = curr_dat['text_value'].value_counts(dropna = False).to_dict()


    res_ls.append([curr_loinc, REQ_LAB_NUM, n_uniq_pats, test_scale, units,
                   nrow, n_pats, units_dict, n_null, n_vals_0, n_vals_non_0,
                   n_unique, num_lt_0, num_mean, num_sd, num_med,
                   num_min, num_max, tens_str, num_vals_str,
                   n_cat_with_value, curr_val_con])
    
res = pd.DataFrame(res_ls, columns = ['loinc', 'REQ_NUM', 'num_uniq_pats',
                                     'test_scale', 'test_units', 'nrow',
                                     'n_pats', 'units_dict', 'n_num_null',
                                     'n_num_vals_0', 'n_num_vals_non_0',
                                     'n_num_unique', 
                                      'n_lt_0', 'mean', 'sd', 'med', 'min', 'max',
                                      'tens_dict', 'hist_dict', 'n_cat_w_val',
                                     'cat_vals_dict'])


res.to_csv(f"{BASE_DIR}/lab_test_data_analysis.tsv", sep = "\t", index = False)

<hr>
The labs results were then manually checked to see which categorical LOINC codes had enough results (negative or positive) for us to be powered to run an analysis.

# Generating disease-pathogen pair data

A python script was used to create a set of data pairing each ICD10 code with each LOINC code. Prior to running, we removed the 7 ICD10 codes from all_icd_af that have no cases in TNX [E14, I64, I84, J46, K07, K10, O81].

  
We call our python script directly from the command file. This python script, trinetx_pairs_single_icd_pub.py can be found elsewhere in this repository. But again we are using sAJ to generate all pairs for each ICD10 code.

<hr> 

```bash
cf.sh
#!/bin/bash

module purge
module load python3/3.10.7

python "${HOME}/code/antigen_research/trinetx/trinetx_pairs_single_icd_pub.py" --icd VAR1

```

<hr> 

```bash
head af
A60_trinetx_pairs       A60
B00_trinetx_pairs       B00
B02_trinetx_pairs       B02
B19_trinetx_pairs       B19
B24_trinetx_pairs       B24
...
```
<hr> 

```bash
submitArrayJobs  -C cf.sh -A af  -W 24:00 -n 1 -M 20000
```

<hr>

<div class="alert alert-block alert-warning">
Many checks were completed to verify that all jobs ran successfully and didn't fail part way through for all parts of TNX data processing. These checks are just not shown here, as they don't seem particular relevant.
</div>

# Patient data / Covariates

## Load in patient data

In [ ]:
# Now look at each of the UKB covariates to see if we can extract data for them in the TNX data
pats = pd.read_csv(f"{BASE_DIR}/patient.csv", sep = ",")

## Sex

In [ ]:
# F          7917303
# M          4177790
# Unknown       7061
pats['sex'].value_counts(dropna = False)

In [ ]:
# Encode like UKB (F = 0)
pats.loc[:, 'sex'] = pats.loc[:, 'sex'].replace({'F' : 0,
                            'M' : 1,
                            'Unknown' : 2})

In [ ]:
# new dataset
# 0    7917303
# 1    4177790
# 2       706
pats['sex'].value_counts(dropna = False)

## BMI - SKIP

In [ ]:
# Only have BMI data for 37.3% of participants, so we can't really
# impute here, so we will just have to drop BMI.

## Age

In [ ]:
# TriNetX data was made available on Feb 13, 2023, so that's the latest
# we should have data for, so calculate age to that date, unless the 
# participant is dead, in which case set age to age at death as that's when
# their exposure ended.
# However, since TNX only has year of birth resolution, people that were born
# in 2023, if we calculated age using year data was made available of 2023, the
# age would be 0, so we will round up the year the data was made available to
# 2024, so the youngest people will be 1 yr old.

from datetime import datetime as dt

def calc_curr_age(curr_pat_row):
    #print(curr_pat_row)
    
    # Year data was released to us (rounded up)
    CURR_YEAR = 2023

    curr_id = curr_pat_row[0]
    curr_yob = curr_pat_row[5]
    curr_death_str = curr_pat_row[7]

    is_dead = pd.notnull(curr_death_str)

    #print(f'{curr_id}: {curr_yob}  -> {curr_death_str}')

    # If they are dead
    if is_dead:

        # Remove trailing decimal (got added at some point)
        curr_death_str = str(int(curr_death_str))

        curr_death = dt.strptime(str(curr_death_str), '%Y%m')

        # Midpoint of year is July 2nd (unless leap year in which case it's July 1)
        # But we only have year and month, so July is midpoint of year

        # Round to next year
        if curr_death.month >= 6:
            death_yr = curr_death.year + 1

        else:
            death_yr = curr_death.year

        curr_age = death_yr - curr_yob

    # Still alive
    else:
        curr_age = CURR_YEAR - curr_yob
        
        
    return curr_age

In [ ]:
# Allows us to show progress bar during apply operations
tqdm_notebook().pandas()

pats['age'] = pats.progress_apply(calc_curr_age, axis = 1)

In [ ]:
# Participants without age values (86,790) will be dropped from dynamically generated
# cohorts when the UKB model we are refitting requires adjustment for age. If the UKB
# model does not adjust for age, then we can keep these people in the cohort.

# Age
# Unique Patients: 12102154
# Mean:   46.352431353723446
# Stdev:  17.465842251088695
# Median: 44.0
# Min:    0.0
# Max:    91.0
# Age = NA: 86790
# Age < 0: 0
# Age = 0: 72
    
print("Age")
print(f"Unique Patients: {len(pats['patient_id'].unique())}")
print(f"Mean:   {pats['age'].mean()}")
print(f"Stdev:  {pats['age'].std()}")
print(f"Median: {pats['age'].median()}")
print(f"Min:    {pats['age'].min()}")
print(f"Max:    {pats['age'].max()}")
print(f"Age = NA: {sum(pats['age'].isna())}")
print(f"Age < 0: {len(pats.loc[pats['age']  < 0, :])}")
print(f"Age = 0: {len(pats.loc[pats['age']  == 0, :])}")


# Scale the age  (divide by 10)
pats['s_age'] = pats['age']/10

## Race/ethnicity

In [ ]:
# White                                        5538331
# Unknown                                      4388318
# Black or African American                    1834228
# Asian                                         280490
# American Indian or Alaska Native               44547
# Native Hawaiian or Other Pacific Islander      16240
pats['race'].value_counts(dropna = False)

In [ ]:
# Not Hispanic or Latino    6303947
# Unknown                   4895691
# Hispanic or Latino         902516
pats['ethnicity'].value_counts(dropna = False)

In [ ]:
# For UKB analysis any mixed race such as 'White and black caribbean' were
# coded as 'other', but for TNX there is a distinct 'hispanic' group, so 
# we will code, any ethnicity that is hispanic/latino as hispanic.
#
# 
# race                                       ethnicity             
# American Indian or Alaska Native           Hispanic or Latino           6219
#                                            Not Hispanic or Latino      22764
#                                            Unknown                     15564
# Asian                                      Hispanic or Latino           3673
#                                            Not Hispanic or Latino     226936
#                                            Unknown                     49881
# Black or African American                  Hispanic or Latino          24876
#                                            Not Hispanic or Latino    1459988
#                                            Unknown                    349364
# Native Hawaiian or Other Pacific Islander  Hispanic or Latino           2729
#                                            Not Hispanic or Latino      10463
#                                            Unknown                      3048
# Unknown                                    Hispanic or Latino         408834
#                                            Not Hispanic or Latino     318647
#                                            Unknown                   3660837
# White                                      Hispanic or Latino         456185
#                                            Not Hispanic or Latino    4265149
#                                            Unknown                    816997
pats.value_counts(['race', 'ethnicity'], dropna = False).sort_index()

In [ ]:
# Changing to if ethnicity is unknown you are classified to race response
# 'Asian, Unknown' -> 'Asian'
# Default to blank but that should get wiped out completely
pats['fin_race'] = 'blank'

pats.loc[((pats['race'] == 'White') & 
          ((pats['ethnicity'] == 'Not Hispanic or Latino') |
          (pats['ethnicity'] == 'Unknown'))), 'fin_race'] = 'white'

pats.loc[((pats['race'] == 'Asian') & 
          ((pats['ethnicity'] == 'Not Hispanic or Latino') |
          (pats['ethnicity'] == 'Unknown'))), 'fin_race'] = 'asian'


pats.loc[((pats['race'] == 'Black or African American') & 
          ((pats['ethnicity'] == 'Not Hispanic or Latino') |
          (pats['ethnicity'] == 'Unknown'))), 'fin_race'] = 'black'

pats.loc[((pats['race'] == 'American Indian or Alaska Native') & 
          ((pats['ethnicity'] == 'Not Hispanic or Latino') |
          (pats['ethnicity'] == 'Unknown'))), 'fin_race'] = 'ai_an'

pats.loc[((pats['race'] == 'Native Hawaiian or Other Pacific Islander') & 
          ((pats['ethnicity'] == 'Not Hispanic or Latino') |
          (pats['ethnicity'] == 'Unknown'))), 'fin_race'] = 'nh_opi'

pats.loc[((pats['race'] == 'Unknown') & 
          ((pats['ethnicity'] == 'Not Hispanic or Latino') |
          (pats['ethnicity'] == 'Unknown'))), 'fin_race'] = 'other'

pats.loc[pats['ethnicity'] == 'Hispanic or Latino', 'fin_race'] = 'hispanic'


In [ ]:
# Counts are good:
# ai_an         22764
# asian        226936
# black       1459988
# hispanic     902516
# nh_opi        10463
# other       5214338
# white       4265149
pats['fin_race'].value_counts(dropna = False).sort_index()

In [ ]:
# UKB variable is called 'ethnic' so let's use that for final code
pats['ethnic'] = pats.loc[:, 'fin_race'].replace({
                                                    'white'    : 0,
                                                    'asian'    : 1,
                                                    'black'    : 2, 
                                                    'hispanic' : 3,
                                                    'ai_an'    : 4,
                                                    'nh_opi'   : 5,
                                                    'other'    : 6

                                                })

In [ ]:
# 0    5082146
# 1     276817
# 2    1809352
# 3     902516
# 4      38328
# 5      13511
# 6    3979484
pats['ethnic'].value_counts(dropna = False).sort_index()

## Townsend Deprivation Index - SKIP

In [ ]:
# No deprivation index available in TNX data

## Number in house - SKIP

In [ ]:
# No number in house is available in TNX data

## Tobacco Use - SKIP

In [ ]:
# No reliable source of tobacco usage could be found in available TNX data

## Alcohol Use - SKIP

In [ ]:
# No reliable source of alcohol usage could be found in available TNX data

## Number of Sex Partners - SKIP

In [ ]:
# No number of sex partners is available in TNX data

## Same-sex Intercourse - SKIP

In [ ]:
# No same-sex intercourse is available in TNX data

## Save covariate file

In [ ]:
pats = pats.set_index('patient_id')
pats = pats.loc[:, ['sex', 'ethnic', 'age']]

In [ ]:
#  86K missing age, we will try to impute those but will also
#              keep a column with NAs for missing ages
# Nan in each columns
# sex           0
# ethnic        0
# age       86790
# get number of NAs in each column
print("Nan in each columns" , pats.isnull().sum(), sep='\n')

In [ ]:
# Create the imputer object
MICE = IterativeImputer(random_state=0)

# do the actual imputation
ret = MICE.fit_transform(pats)

# Convert imputed data from np array to df
ret_df = pd.DataFrame(ret)

# Reset the column names and indices
ret_df.columns = pats.columns
ret_df.index = pats.index

ret_df = ret_df.loc[:, ['age']].copy(deep = True)
ret_df.columns = ['imp_age']

pats = pats.merge(ret_df, left_index = True, right_index = True, how = 'left')

In [ ]:
# New dataset: 86K missing age, but now we have imputed age with nothing missing
# Nan in each columns
# sex            0
# ethnic         0
# age        86790
# imp_age        0
# get number of NAs in each column
print("Nan in each columns" , pats.isnull().sum(), sep='\n')

In [ ]:
pats = pats.reset_index(drop = False)

In [ ]:
# Sex:
# 0    7917303
# 1    4177790
# 2       7061
# Name: sex, dtype: int64
# ==================================
# Ethnic:
# 0    5082146
# 1     276817
# 2    1809352
# 3     902516
# 4      38328
# 5      13511
# 6    3979484
# Name: ethnic, dtype: int64
# ==================================
# Age
# Unique Patients: 12102154
# Mean:   46.352431353723446
# Stdev:  17.465842251088695
# Median: 44.0
# Min:    0.0
# Max:    91.0
# Age = NA: 86790
# Age < 0: 0
# Age = 0: 72
# ==================================
# Imp Age
# Unique Patients: 12102154
# Mean:   46.35241817051056
# Stdev:  17.404778331928817
# Median: 44.0
# Min:    0.0
# Max:    91.0
# Age = NA: 0
# Age < 0: 0
# Age = 0: 72

print(f"Sex:")
print(pats['sex'].value_counts(dropna = False).sort_index())
print("==================================")


print(f"Ethnic:")
print(pats['ethnic'].value_counts(dropna = False).sort_index())
print("==================================")


print("Age")
print(f"Unique Patients: {len(pats['patient_id'].unique())}")
print(f"Mean:   {pats['age'].mean()}")
print(f"Stdev:  {pats['age'].std()}")
print(f"Median: {pats['age'].median()}")
print(f"Min:    {pats['age'].min()}")
print(f"Max:    {pats['age'].max()}")
print(f"Age = NA: {sum(pats['age'].isna())}")
print(f"Age < 0: {len(pats.loc[pats['age']  < 0, :])}")
print(f"Age = 0: {len(pats.loc[pats['age']  == 0, :])}")
print("==================================")


print("Imp Age")
print(f"Unique Patients: {len(pats['patient_id'].unique())}")
print(f"Mean:   {pats['imp_age'].mean()}")
print(f"Stdev:  {pats['imp_age'].std()}")
print(f"Median: {pats['imp_age'].median()}")
print(f"Min:    {pats['imp_age'].min()}")
print(f"Max:    {pats['imp_age'].max()}")
print(f"Age = NA: {sum(pats['imp_age'].isna())}")
print(f"Age < 0: {len(pats.loc[pats['imp_age']  < 0, :])}")
print(f"Age = 0: {len(pats.loc[pats['imp_age']  == 0, :])}")

In [ ]:
# Scale ages before saving output
pats.loc[:, 'age'] = pats.loc[:, 'age'] / 10
pats.loc[:, 'imp_age'] = pats.loc[:, 'imp_age'] / 10

In [ ]:
# Save the file
pats.to_csv(f"{BASE_DIR}/procd_data/procd_covs.tsv", sep = '\t', index = False)

# Healthy Pregnancies

In [ ]:
# We need to get a list of the particpants who have had a healthy birth as our control
# group for any ICD10 O codes.
HEALTHY_PREGNANCY_ICD_CODES = ["O80", "O81", "O82", "O83", "O84"]

icd_dir = f'{BASE_DIR}/icd_data'


diag_cols = ['pat_id', 'enc_id', 'vocab', 'full_code',
             'principal_diag_indicator', 'admit_diag',
             'reason_for_visit', 'date', 'derived_by_tri',
             'source_id', 'icd_3_char', 'icd_sub_cat']

dtype_dict = {'pat_id': str, 'enc_id': str, 'vocab': str, 'full_code': str,
              'principal_diag_indicator': str, 'admit_diag': str,
              'reason_for_visit': str, 'date': str,
              'derived_by_tri': str, 'source_id': str, 'icd_3_char': str,
              'icd_sub_cat': str}

fin_dat = pd.DataFrame(columns = diag_cols)

for curr_icd in tqdm(HEALTHY_PREGNANCY_ICD_CODES):
    icd_fn = f"{icd_dir}/{curr_icd}_only.csv"
    
    if not os.path.exists(icd_fn):
        print(f"No ICD data file for {curr_icd}... skipping")
        continue
        
    curr_dat = pd.read_csv(icd_fn, names = diag_cols, dtype = dtype_dict)
    fin_dat = pd.concat([fin_dat, curr_dat], axis = 0)

In [ ]:
fin_dat = fin_dat.drop_duplicates('pat_id').loc[:, ['pat_id']]

In [ ]:
# Save the file out
fin_dat.to_csv(f'{BASE_DIR}/procd_data/healthy_pregnancy_data.tsv',
               sep = '\t', index = False)